In [1]:
from sdssdb.peewee.sdss5db import targetdb

# connect to targetdb
targetdb.database.connect_from_parameters(user='sdss_user', password='4-replicant',
                                          host='localhost',
                                          port=7500)

True

In [2]:
from peewee import *
import numpy as np

##MEG & JON: 3 changes to be made
# set search radius based on field size for APO or LCO
observatory = 'LCO'
if observatory == 'APO':
    r_search = 1.49
else:
    r_search = 0.95
skies = 50
# specify the field center and position angle
#MEG & JON: CHANGE FOR FIELD
#from: https://wiki.sdss.org/display/BHM/Reverberation+Mapping
##XMM LSS #s
# racen = 35.708333
# deccen = -5.050000
# fieldname = "XMMLSS"
##SDSS-RM
# racen = 213.7041666
# deccen = 53.083333
# fieldname = "RM"
# ##COSMOS
# racen = 10.0*15
# deccen = 2.20
# fieldname = "COSMOS"


In [3]:
## SOUTHERN FIELDS

# #skies: best
racen = 52.6483
deccen = -28.1
fieldname = "DESCNC3"
# cartons = ['bhm_csc_boss', 'bhm_spiders_agn_lsdr8','bhm_rm_known_spec', 'bhm_rm_core','bhm_rm_var','bhm_rm_ancillary','ops_std_boss_gdr2']

#skies: good
# racen = 90.0
# deccen = -66.5605
# fieldname = "SCVZ"
# cartons = ['bhm_rm_known_spec', 'bhm_rm_core','bhm_rm_var', 'bhm_spiders_agn_sep','bhm_rm_ancillary','ops_std_boss_gdr2']

# ##skies: best
# racen = 9.45
# deccen = -44
# fieldname = "ELAIS"
# cartons = ['bhm_csc_boss', 'bhm_spiders_agn_lsdr8','bhm_rm_known_spec', 'bhm_rm_core','bhm_rm_var','bhm_rm_ancillary','ops_std_boss_gdr2']

In [4]:
cartons = ['bhm_rm_known_spec', 'bhm_rm_var', 'bhm_rm_core','bhm_rm_ancillary','bhm_rm_xrayqso']#, 'ops_std_boss_gdr2']#APO:'ops_std_eboss', 'ops_sky_boss' ]#['bhm_csc_boss', 'bhm_spiders_agn_lsdr8'] #['bhm_rm_known_spec', 'bhm_rm_core','bhm_rm_var', 'bhm_spiders_agn_sep','bhm_rm_ancillary','ops_std_boss_gdr2']#'ops_sky_apogee',
plan = '1.0.12'
# get all of the targets in the commissioning carton near the field center
all_targs = (targetdb.Target.select(targetdb.Target.catalogid,
                                   targetdb.Target.ra,
                                   targetdb.Target.dec,
                                   targetdb.Target.pk,
                                   targetdb.CartonToTarget.priority,
                                   targetdb.CartonToTarget.value,
                                   targetdb.Cadence.label,
                                   targetdb.Carton.carton,
                                   targetdb.CartonToTarget.pk,
                                   targetdb.Magnitude.g,
                                   targetdb.Magnitude.r,
                                   targetdb.Magnitude.i,
                                   targetdb.Magnitude.z,
                                   targetdb.Magnitude.bp,
                                   targetdb.Magnitude.gaia_g,
                                   targetdb.Magnitude.rp,
                                   targetdb.Magnitude.j,
                                   targetdb.Magnitude.h,
                                   targetdb.Magnitude.k,
                                   targetdb.Category.label,
                                   targetdb.Carton.program,
                                   targetdb.Instrument.label,
                                   targetdb.Version.plan)
                    .join(targetdb.CartonToTarget)
                    .join(targetdb.Cadence, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Carton)
                    .join(targetdb.Version, JOIN.LEFT_OUTER)
                    .switch(targetdb.Carton)
                    .join(targetdb.Category)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Magnitude, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Instrument)
                    .where((targetdb.Carton.carton.in_(cartons)) &
                           (targetdb.Target.cone_search(racen, deccen, r_search)) &
                           (targetdb.Version.plan == plan)))

ops_carton = ['ops_sky_boss_good', 'ops_sky_boss_best']
ops_plan = '0.5.9'
ops_query = (targetdb.Target.select(targetdb.Target.catalogid,
                                   targetdb.Target.ra,
                                   targetdb.Target.dec,
                                   targetdb.Target.pk,
                                   targetdb.CartonToTarget.priority,
                                   targetdb.CartonToTarget.value,
                                   targetdb.Cadence.label,
                                   targetdb.Carton.carton,
                                   targetdb.CartonToTarget.pk,
                                   targetdb.Magnitude.g,
                                   targetdb.Magnitude.r,
                                   targetdb.Magnitude.i,
                                   targetdb.Magnitude.z,
                                   targetdb.Magnitude.bp,
                                   targetdb.Magnitude.gaia_g,
                                   targetdb.Magnitude.rp,
                                   targetdb.Magnitude.j,
                                   targetdb.Magnitude.h,
                                   targetdb.Magnitude.k,
                                   targetdb.Category.label,
                                   targetdb.Carton.program,
                                   targetdb.Instrument.label,
                                   targetdb.Version.plan)
                    .join(targetdb.CartonToTarget)
                    .join(targetdb.Cadence, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Carton)
                    .join(targetdb.Version, JOIN.LEFT_OUTER)
                    .switch(targetdb.Carton)
                    .join(targetdb.Category)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Magnitude, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Instrument)
                    .where((targetdb.Carton.carton.in_(ops_carton)) &
                           (targetdb.Target.cone_search(racen, deccen, r_search)) &
                           (targetdb.Version.plan == ops_plan)))

std_carton = ['ops_std_boss_gdr2']#['ops_std_eboss']#, 'ops_std_boss_lsdr10', 'ops_std_boss_gdr2']#
std_plan = ['1.0.13']#,'1.0.5','1.0.13'] #'0.5.0'
std_query = (targetdb.Target.select(targetdb.Target.catalogid,
                                   targetdb.Target.ra,
                                   targetdb.Target.dec,
                                   targetdb.Target.pk,
                                   targetdb.CartonToTarget.priority,
                                   targetdb.CartonToTarget.value,
                                   targetdb.Cadence.label,
                                   targetdb.Carton.carton,
                                   targetdb.CartonToTarget.pk,
                                   targetdb.Magnitude.g,
                                   targetdb.Magnitude.r,
                                   targetdb.Magnitude.i,
                                   targetdb.Magnitude.z,
                                   targetdb.Magnitude.bp,
                                   targetdb.Magnitude.gaia_g,
                                   targetdb.Magnitude.rp,
                                   targetdb.Magnitude.j,
                                   targetdb.Magnitude.h,
                                   targetdb.Magnitude.k,
                                   targetdb.Category.label,
                                   targetdb.Carton.program,
                                   targetdb.Instrument.label,
                                   targetdb.Version.plan)
                    .join(targetdb.CartonToTarget)
                    .join(targetdb.Cadence, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Carton)
                    .join(targetdb.Version, JOIN.LEFT_OUTER)
                    .switch(targetdb.Carton)
                    .join(targetdb.Category)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Magnitude, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Instrument)
                    .where((targetdb.Carton.carton.in_(std_carton)) &
                           (targetdb.Target.cone_search(racen, deccen, r_search)) &
                           (targetdb.Version.plan == std_plan)))


AQMES_carton = [ 'bhm_spiders_agn_lsdr10', 'bhm_csc_boss']#'bhm_spiders_agn_sep'
AQMES_plan = '1.0.37'
AQMES_query = (targetdb.Target.select(targetdb.Target.catalogid,
                                   targetdb.Target.ra,
                                   targetdb.Target.dec,
                                   targetdb.Target.pk,
                                   targetdb.CartonToTarget.priority,
                                   targetdb.CartonToTarget.value,
                                   targetdb.Cadence.label,
                                   targetdb.Carton.carton,
                                   targetdb.CartonToTarget.pk,
                                   targetdb.Magnitude.g,
                                   targetdb.Magnitude.r,
                                   targetdb.Magnitude.i,
                                   targetdb.Magnitude.z,
                                   targetdb.Magnitude.bp,
                                   targetdb.Magnitude.gaia_g,
                                   targetdb.Magnitude.rp,
                                   targetdb.Magnitude.j,
                                   targetdb.Magnitude.h,
                                   targetdb.Magnitude.k,
                                   targetdb.Category.label,
                                   targetdb.Carton.program,
                                   targetdb.Instrument.label,
                                   targetdb.Version.plan)
                    .join(targetdb.CartonToTarget)
                    .join(targetdb.Cadence, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Carton)
                    .join(targetdb.Version, JOIN.LEFT_OUTER)
                    .switch(targetdb.Carton)
                    .join(targetdb.Category)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Magnitude, JOIN.LEFT_OUTER)
                    .switch(targetdb.CartonToTarget)
                    .join(targetdb.Instrument)
                    .where((targetdb.Carton.carton.in_(AQMES_carton)) &
                           (targetdb.Target.cone_search(racen, deccen, r_search)) &
                           (targetdb.Version.plan == AQMES_plan)))

# # grab the results
catalogid1, ra1, dec1, target_pk1, priority1, value1, cadences1, carton1, carton_to_target_pk1, g1, r1, i1, z1, bp1, gaia_g1, rp1, j1, h1,k1, category1, program1, instrument1,plan1  = map(list, zip(*list(all_targs.tuples())))
catalogid1 = np.array(catalogid1, dtype=np.int64)
ra1 = np.array(ra1)
dec1 = np.array(dec1)
target_pk1 = np.array(target_pk1, dtype=np.int64)
priority1 = np.array(priority1)
value1 = np.array(value1)
carton1 = np.array(carton1)
carton_to_target_pk1 = np.array(carton_to_target_pk1)
magnitudes1 = np.zeros((len(g1),10))
magnitudes1[:, 0] = g1
magnitudes1[:, 1] = r1
magnitudes1[:, 2] = i1
magnitudes1[:, 3] = z1
magnitudes1[:, 4] = bp1
magnitudes1[:, 5] = gaia_g1
magnitudes1[:, 6] = rp1
magnitudes1[:, 7] = j1
magnitudes1[:, 8] = h1
magnitudes1[:, 9] = k1
category1 = np.array(category1)
program1 = np.array(program1)
instrument1 = np.array(instrument1)
plan1 = np.array(plan1)
#tag1 = np.full(len(catalogid1), '1.0.12')
# # grab the results, but for the sky stuff
catalogid2, ra2, dec2, target_pk2, priority2, value2, cadences2, carton2, carton_to_target_pk2, g2, r2, i2, z2, bp2, gaia_g2, rp2, j2, h2, k2, category2, program2, instrument2, plan2 = map(list, zip(*list(ops_query.tuples())))
catalogid2 = np.array(catalogid2, dtype=np.int64)
ra2 = np.array(ra2)
dec2 = np.array(dec2)
target_pk2 = np.array(target_pk2, dtype=np.int64)
priority2 = np.array(priority2)
value2 = np.array(value2)
carton2 = np.array(carton2)
carton_to_target_pk2 = np.array(carton_to_target_pk2)
magnitudes2 = np.zeros((len(g2),10))
magnitudes2[:, 0] = g2
magnitudes2[:, 1] = r2
magnitudes2[:, 2] = i2
magnitudes2[:, 3] = z2
magnitudes2[:, 4] = bp2
magnitudes2[:, 5] = gaia_g2
magnitudes2[:, 6] = rp2
magnitudes2[:, 7] = j2
magnitudes2[:, 8] = h2
magnitudes2[:, 9] = k2
category2 = np.array(category2)
program2 = np.array(program2)
instrument2 = np.array(instrument2)
plan2 = np.array(plan2)

catalogid3, ra3, dec3, target_pk3, priority3, value3, cadences3, carton3, carton_to_target_pk3, g3, r3, i3, z3, bp3, gaia_g3, rp3, j3, h3, k3, category3, program3, instrument3, plan3 = map(list, zip(*list(std_query.tuples())))
catalogid3 = np.array(catalogid3, dtype=np.int64)
ra3 = np.array(ra3)
dec3 = np.array(dec3)
target_pk3 = np.array(target_pk3, dtype=np.int64)
priority3 = np.array(priority3)
value3 = np.array(value3)
carton3 = np.array(carton3)
carton_to_target_pk3 = np.array(carton_to_target_pk3)
magnitudes3 = np.zeros((len(g3),10))
magnitudes3[:, 0] = g3
magnitudes3[:, 1] = r3
magnitudes3[:, 2] = i3
magnitudes3[:, 3] = z3
magnitudes3[:, 4] = bp3
magnitudes3[:, 5] = gaia_g3
magnitudes3[:, 6] = rp3
magnitudes3[:, 7] = j3
magnitudes3[:, 8] = h3
magnitudes3[:, 9] = k3
category3 = np.array(category3)
program3 = np.array(program3)
instrument3 = np.array(instrument3)
plan3 = np.array(plan3)

catalogid4, ra4, dec4, target_pk4, priority4, value4, cadences4, carton4, carton_to_target_pk4, g4, r4, i4, z4, bp4, gaia_g4, rp4, j4, h4, k4, category4, program4, instrument4, plan4 = map(list, zip(*list(AQMES_query.tuples())))
catalogid4 = np.array(catalogid4, dtype=np.int64)
ra4 = np.array(ra4)
dec4 = np.array(dec4)
target_pk4 = np.array(target_pk4, dtype=np.int64)
priority4 = np.array(priority4)
value4 = np.array(value4)
carton4 = np.array(carton4)
carton_to_target_pk4 = np.array(carton_to_target_pk4)
magnitudes4 = np.zeros((len(g4),10))
magnitudes4[:, 0] = g4
magnitudes4[:, 1] = r4
magnitudes4[:, 2] = i4
magnitudes4[:, 3] = z4
magnitudes4[:, 4] = bp4
magnitudes4[:, 5] = gaia_g4
magnitudes4[:, 6] = rp4
magnitudes4[:, 7] = j4
magnitudes4[:, 8] = h4
magnitudes4[:, 9] = k4
category4 = np.array(category4)
program4 = np.array(program4)
instrument4 = np.array(instrument4)
plan4 = np.array(plan4)


# # #append together, only 1-3
# catalogid = np.concatenate([catalogid1, catalogid2, catalogid3], axis=0)
# ra = np.concatenate([ra1, ra2,ra3], axis=0)
# dec = np.concatenate([dec1, dec2, dec3], axis=0)
# target_pk = np.concatenate([target_pk1, target_pk2,target_pk3], axis=0)
# priority = np.concatenate([priority1, priority2,priority3], axis=0)
# value = np.concatenate([value1, value2, value3], axis=0)
# carton = np.concatenate([carton1, carton2, carton3], axis=0)
# carton_to_target_pk = np.concatenate([carton_to_target_pk1, carton_to_target_pk2, carton_to_target_pk3], axis=0)
# g = np.concatenate([g1, g2, g3], axis=0)
# r = np.concatenate([r1, r2, r3], axis=0)
# i = np.concatenate([i1, i2, i3], axis=0)
# z = np.concatenate([z1, z2, z3], axis=0)
# bp = np.concatenate([bp1, bp2, bp3], axis=0)
# gaia_g = np.concatenate([gaia_g1, gaia_g2, gaia_g3], axis=0)
# rp = np.concatenate([rp1, rp2, rp3], axis=0)
# j = np.concatenate([j1, j2, j3], axis=0)
# h = np.concatenate([h1, h2, h3], axis=0)
# k = np.concatenate([k1, k2, k3], axis=0)

# magnitudes = np.concatenate([magnitudes1, magnitudes2,magnitudes3], axis=0)
# program = np.concatenate([program1, program2,program3], axis=0)
# category = np.concatenate([category1, category2, category3], axis=0)
# instrument = np.concatenate([instrument1, instrument2,instrument3], axis=0)
# plan = np.concatenate([plan1, plan2, plan3], axis=0)

# tag = np.full(len(catalogid), '1.0.12')

# #append together, for AQMES too
catalogid = np.concatenate([catalogid1, catalogid2, catalogid3, catalogid4], axis=0)
ra = np.concatenate([ra1, ra2,ra3, ra4], axis=0)
dec = np.concatenate([dec1, dec2, dec3, dec4], axis=0)
target_pk = np.concatenate([target_pk1, target_pk2,target_pk3, target_pk4], axis=0)
priority = np.concatenate([priority1, priority2,priority3, priority4], axis=0)
value = np.concatenate([value1, value2, value3, value4], axis=0)
carton = np.concatenate([carton1, carton2, carton3, carton4], axis=0)
carton_to_target_pk = np.concatenate([carton_to_target_pk1, carton_to_target_pk2, carton_to_target_pk3, carton_to_target_pk4], axis=0)
g = np.concatenate([g1, g2, g3, g4], axis=0)
r = np.concatenate([r1, r2, r3, r4], axis=0)
i = np.concatenate([i1, i2, i3, i4], axis=0)
z = np.concatenate([z1, z2, z3, z4], axis=0)
bp = np.concatenate([bp1, bp2, bp3, bp4], axis=0)
gaia_g = np.concatenate([gaia_g1, gaia_g2, gaia_g3, gaia_g4], axis=0)
rp = np.concatenate([rp1, rp2, rp3, rp4], axis=0)
j = np.concatenate([j1, j2, j3, j4], axis=0)
h = np.concatenate([h1, h2, h3, h4], axis=0)
k = np.concatenate([k1, k2, k3, k4], axis=0)

magnitudes = np.concatenate([magnitudes1, magnitudes2,magnitudes3, magnitudes4], axis=0)
program = np.concatenate([program1, program2,program3, program4], axis=0)
category = np.concatenate([category1, category2, category3, category4], axis=0)
instrument = np.concatenate([instrument1, instrument2,instrument3, instrument4], axis=0)
plan = np.concatenate([plan1, plan2, plan3, plan4], axis=0)

tag = np.full(len(catalogid), '1.0.12')


# # specify the instrument for each target
# # instrument = np.zeros(len(carton), dtype='<U10')

# # #MEG & JON: CHANGE FOR FIELD
# # instrument[carton == 'ops_sky_boss'] = 'BOSS'
# # instrument[carton == 'ops_std_eboss'] = 'BOSS'
# # instrument[carton == 'bhm_rm_known_spec'] = 'BOSS'
# # instrument[carton == 'bhm_rm_core'] = 'BOSS'
# # instrument[carton == 'bhm_rm_var'] = 'BOSS'
# # instrument[carton == 'bhm_rm_ancillary'] = 'BOSS'

In [5]:
plan.shape

(4609,)

In [6]:
len(catalogid)

4609

In [7]:
ks = (np.array(category) == 'science')
len(np.unique(np.array(catalogid)[ks]))

762

In [8]:
1813-1772

41

In [9]:
## CHECKING OUT X_RAY STUFF
print('field: ', fieldname)
print("unique cartons:", np.unique(carton))
# print("bhm_csc_boss:", len(np.where((carton == np.unique(carton)[0]))[0]))
# print("bhm_spiders_agn_lsdr8:", len(np.where((carton == np.unique(carton)[1]))[0]))
print("tot new: ", len(catalogid))
print("tot new unique: ", len(np.unique(catalogid)))

field:  DESCNC3
unique cartons: ['bhm_csc_boss' 'bhm_rm_ancillary' 'bhm_rm_core' 'bhm_rm_known_spec'
 'bhm_rm_var' 'bhm_rm_xrayqso' 'bhm_spiders_agn_lsdr10'
 'ops_sky_boss_best' 'ops_sky_boss_good' 'ops_std_boss_gdr2']
tot new:  4609
tot new unique:  2617


In [10]:
print("skies:", len(np.where((carton == 'ops_sky_boss_good'))[0])) #may need to change to _good or _best
print("stds:", len(np.where((carton == 'ops_std_boss_gdr2'))[0]))
#print("science- erosita carton SCVZ ONLY:", len(np.where((carton == 'bhm_spiders_agn_sep'))[0]))
print("science targs: ", len(np.where((category == 'science'))[0]))
print("unique cartons:", np.unique(carton))

skies: 1712
stds: 143
science targs:  1538
unique cartons: ['bhm_csc_boss' 'bhm_rm_ancillary' 'bhm_rm_core' 'bhm_rm_known_spec'
 'bhm_rm_var' 'bhm_rm_xrayqso' 'bhm_spiders_agn_lsdr10'
 'ops_sky_boss_best' 'ops_sky_boss_good' 'ops_std_boss_gdr2']


In [35]:
# #S-CVZ NEW TARGET DESIGN: EDIT KNOWN_SPEC AND SPIDERS, INCREASE PRIORITY (DEPRIORITIZE)
# depriori = ((carton == 'bhm_rm_known_spec')|(carton == 'bhm_spiders_agn_sep'))
# priority[depriori] += 1000
# priority[depriori]
#np.unique(priority[(carton == 'bhm_rm_known_spec' )])

array([2001, 2041])

In [11]:
from astropy.table import Table, Column
tab = Table()
tab.add_column(Column(name="catalogid", data=catalogid, dtype='int64'))
tab.add_column(Column(name="ra", data=ra, dtype='float32'))
tab.add_column(Column(name="dec", data=dec, dtype='float32'))
tab.add_column(Column(name="target_pk", data=target_pk, dtype='int64'))
tab.add_column(Column(name="priority", data=priority, dtype='float32'))
tab.add_column(Column(name="value", data=value, dtype='float32'))
tab.add_column(Column(name="carton", data=carton))
tab.add_column(Column(name="instrument", data=instrument, dtype='<U10'))
tab.add_column(Column(name="carton_to_target_pk", data=carton_to_target_pk, dtype='int64'))
tab.add_column(Column(name="g", data=g, dtype='float32'))
tab.add_column(Column(name="r", data=r, dtype='float32'))
tab.add_column(Column(name="i", data=i, dtype='float32'))
tab.add_column(Column(name="z", data=z, dtype='float32'))
tab.add_column(Column(name="bp", data=bp, dtype='float32'))
tab.add_column(Column(name="gaia_g", data=gaia_g, dtype='float32'))
tab.add_column(Column(name="rp", data=rp, dtype='float32'))
tab.add_column(Column(name="j", data=j, dtype='float32'))
tab.add_column(Column(name="h", data=h, dtype='float32'))
tab.add_column(Column(name="k", data=k, dtype='float32'))
tab.add_column(Column(name="category", data=category))
tab.add_column(Column(name="tag", data=tag, dtype='<U10'))
tab.add_column(Column(name="plan", data=plan, dtype='<U10'))
tab.add_column(Column(name="program", data=program))
tab.write("BHM_cartons_{}_v1.fits".format(fieldname), format='fits', overwrite=True)

In [80]:
1413-1374

39

In [79]:
from astropy.table import Table, Column
data = Table.read("./BHM_cartons_XMMLSS_v1.fits", format='fits')
np.unique(data['catalogid'][(data['category'] == 'science')])

63050395065900021
63050395065900064
63050395065900084
63050395065900085
63050395065900097
63050395065900106
63050395065900107
63050395065900123
63050395065900129
63050395065900158
63050395065900165


In [31]:
np.unique(data['value'])

1000.0
nan


In [43]:
np.unicode("0.5.3")

[WARNING]: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here. (DeprecationWarning)


'0.5.3'

In [9]:
#catalogdb for redshifts?

import pandas as pd
import numpy as np
from sdssdb.peewee.sdss5db import catalogdb
# connect to catalogdb
catalogdb.database.connect_from_parameters(user='sdss_user', password='4-replicant',
                                          host='localhost',
                                          port=7500)

[WARNING]: catalog_to_legacy_survey_dr10a: cannot find related table 'legacy_survey_dr10a' (SdssdbUserWarning)


True

In [ ]:
catalogdb.SDSS

In [15]:

query = (catalogdb..select(catalogdb.SDSS_DR16_SpecObj.ra,
                                       catalogdb.SDSS_DR16_SpecObj.dec,
                                       catalogdb.SDSS_DR16_SpecObj.z)
                                .where(catalogdb.SDSS_DR16_SpecObj.cone_search(racen,
                                                                      deccen,
                                                                      r_search)))


# get the ras, decs and DR2 IDs for the query
ras, decs, zs = map(list, zip(*list(query.tuples())))

In [16]:
len(decs)

3686